In [1]:
import json, jsonschema, jwt, datetime, requests, uuid

## Utility functions

In [2]:
def urn_uuid(uuid):
    return f'urn:uuid:{uuid}'

In [3]:
def jwt_serialization(credential, secret_key, audience):
    payload = {
        "iss": credential["issuer"],  # Issuer
        "aud": audience,  # Audience
        "exp": datetime.datetime.utcnow() + datetime.timedelta(days=1),  # Expiration time
        "nbf": datetime.datetime.utcnow(),  # Not before
        "iat": datetime.datetime.utcnow(),  # Issued at
    }
    if "VerifiableCredential" in credential["type"]:
        payload["sub"] = credential["credentialSubject"]["id"]
        payload["vc"] = credential
    elif "VerifiablePresentation" in credential["type"]:
        payload["vp"] = credential
    return jwt.encode(payload, secret_key, algorithm='HS256')

## A example Merit credential

In [4]:
merit = json.load(open("test_merit.json", "r"))

merit

{'id': 'dc30880d-2d4b-447c-905e-b79075aea95b',
 'acceptedAt': '2023-07-17T08:52:19.489566Z',
 'active': True,
 'activenessFailures': None,
 'authorizedAt': '2023-07-17T08:52:15.960677Z',
 'completed': True,
 'completenessFailures': None,
 'createdAt': '2023-07-17T15:52:04.3018Z',
 'description': 'Template for admin merit',
 'fields': [{'canShare': False,
   'description': "Use this to store recipient's first name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'First Name',
   'templateFieldID': '1b397a27-14a0-4d43-a36f-6006611f4073',
   'templateFieldLineage': ['3dc3f769-70cc-4e7e-81cc-ed103f9eb601'],
   'updatedAt': '2023-07-17T15:52:19.496539Z',
   'validationErrors': None,
   'value': 'Hank'},
  {'canShare': False,
   'description': "Use this to store recipient's last name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'Last Name',
   

## A verifiable credential generated from the example Merit credential

In [7]:
id = urn_uuid(merit["id"])
issuer = urn_uuid(merit["issuer"]["id"])
subject = urn_uuid(merit["recipient"]["id"])

verifiable_credential = {
    "@context": [
        "https://www.w3.org/2018/credentials/v1",
        "https://www.w3.org/2018/credentials/examples/v1",
        "https://schema.org/"
    ],
    "issuer": issuer,
    "id": id,
    "type": ["VerifiableCredential", merit["name"]],
    "credentialSubject": {
        "id": subject,
        "merit": { field["name"]: field["value"] for field in merit["fields"] },
    },
    "issuanceDate": merit["createdAt"],
    "credentialStatus": {
        "id": f'https://vc.merit.com/status/{requests.utils.quote(id)}',
        "type": "MeritCredentialStatusList2024",
        "statusPurpose": merit['state']['name'],
        "occurredAt": merit['state']['occurredAt']
    }
}

verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://vc.merit.com/status/urn%3Auuid%3Adc30880d-2d

## JSON Schema validation of the verifiable credential

In [13]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@context": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },
        "id": {
            "type": "string"
        },
        "type": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "contains": {
                "const": "VerifiableCredential"
            }
        },
        "issuer": {
            "type": "string"
        },
        "issuanceDate": {
            "type": "string",
            "format": "date-time"
        },
        "credentialSubject": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                }
            },
            "required": ["id"]
        },
        "credentialStatus": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                },
                "type": {
                    "type": "string"
                },
                "statusPurpose": {
                    "type": "string"
                },
                "occurredAt": {
                    "type": "string",
                    "format": "date-time"
                }
            },
            "required": ["id"]
        },
        "proof": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string"
                },
                "created": {
                    "type": "string",
                    "format": "date-time"
                },
                "proofValue": {
                    "type": "string"
                },
                "verificationMethod": {
                    "type": "string"
                },
                "jws": {
                    "type": "string"
                }
            },
            "required": ["type", "created", "proofValue"]
        }
    },
    "required": ["@context", "type", "issuer", "issuanceDate", "credentialSubject", "credentialStatus"]
}

In [14]:
try:
    jsonschema.validate(instance=verifiable_credential, schema=schema)
    print("Credential is valid.")
except jsonschema.exceptions.ValidationError as e:
    print("Credential is not valid:", e)

Credential is valid.


## JSON Web Token (JWT) serialization of the verifiable credential

In [15]:
vc_jwt_token = jwt_serialization(verifiable_credential, "secret", "https://verifier.example.gov")

In [16]:
jwt.get_unverified_header(vc_jwt_token)

{'alg': 'HS256', 'typ': 'JWT'}

In [17]:
decoded_payload = jwt.decode(vc_jwt_token, "secret", algorithms='HS256', audience='https://verifier.example.gov')
decoded_payload['vc']

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://vc.merit.com/status/urn%3Auuid%3Adc30880d-2d

## Adding geofencing data into the credentialSubject of the verifiable credential

In [18]:
verifiable_credential["credentialSubject"]["eligibleRegion"] = {
    "type": "GeoCircle",
    "geoMidpoint": {
        "type": "GeoCoordinates",
        "latitude": "38.88508",
        "longitude": "-77.02295"
    },
    "geoRadius": "50"
}

In [19]:
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'},
  'eligibleRegion': {'type': 'GeoCircle',
   'geoMidpoint': {'type': 'GeoCoordinates',
    'latitude': '38.88508',
    'longitud

In [20]:
vc_jwt_token = jwt_serialization(verifiable_credential, "secret", "https://verifier.example.gov")

## Generating a geolocation verifiable credential

In [21]:
geolocation_vc_uuid = uuid.uuid4().hex
geolocation_vc = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': subject,
    'id': f'urn:uuid:{geolocation_vc_uuid}',
    'type': ['VerifiableCredential', 'RealtimeGeolocation'],
    'credentialSubject' : {
        'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
        'geo': {
            'type': 'GeoCoordinates',
            'latitude': '38.88460',
            'longitude': '-77.02273'
        }
    },
    'issuanceDate': datetime.datetime.utcnow().isoformat() + 'Z'
}

In [22]:
geolocation_vc_jwt_token = jwt_serialization(geolocation_vc, "secret", "https://verifier.example.gov")

## Generating a verifiable presentation combining the geofenced credential with the geolocation credential

In [23]:
vp_uuid = uuid.uuid4().hex
verifiable_presentation = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': subject,
    'id': f'urn:uuid:{vp_uuid}',
    'type': ['VerifiablePresentation'],
    'verifiableCredential': [
        vc_jwt_token,
        geolocation_vc_jwt_token
    ]
}

In [24]:
verifiable_presentation

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:4fbdde7e48ee4bc18aa9c01b808fca52',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': ['eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTcxNTQ1LCJuYmYiOjE2OTQ4ODUxNDUsImlhdCI6MTY5NDg4NTE0NSwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MSIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwiaWQiOiJ1cm46dXVpZDpkYzMwODgwZC0yZDRiLTQ0N2MtOTA1ZS1iNzkwNzVhZWE5NWIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiQWRtaW4gbWVyaXQiXSwiY3Jl

In [25]:
geoaware_vp_jwt_token = jwt_serialization(verifiable_presentation, "secret", "https://verifier.example.gov")

In [26]:
decoded_payload = jwt.decode(geoaware_vp_jwt_token, "secret", algorithms='HS256', audience='https://verifier.example.gov')
decoded_vp = decoded_payload['vp']
decoded_vp["verifiableCredential"] = [ 
    jwt.decode(payload, "secret", algorithms='HS256', audience='https://verifier.example.gov')["vc"] for payload in decoded_vp["verifiableCredential"] 
]
decoded_vp

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:4fbdde7e48ee4bc18aa9c01b808fca52',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': [{'@context': ['https://www.w3.org/2018/credentials/v1',
    'https://www.w3.org/2018/credentials/examples/v1',
    'https://schema.org/'],
   'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
   'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
   'type': ['VerifiableCredential', 'Admin merit'],
   'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
    'merit': {'First Name': 'Hank',
     'Last Name': 'Lindgren',
     'Email': 'oleg+user2@gomerits.com',
     'Issuing Org Name': 'Merit International, Inc.',
     'Org UUID': 'user2',
     'Admin Phone Number': '6317070926',
     'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8